Copyright &copy; 1997-2013 The R Core Team

In [1]:
## being a 'builtin' function is not the same as being in base
ls.base <- ls("package:base", all=TRUE)
base.is.f <- sapply(ls.base, function(x) is.function(get(x)))
cat("\nNumber of base objects:\t\t", length(ls.base),
    "\nNumber of functions in base:\t", sum(base.is.f),
    "\n\t starting with 'is.' :\t  ",
    sum(grepl("^is\\.", ls.base[base.is.f])), "\n", sep = "")
## 0.14  : 31
## 0.50  : 33
## 0.60  : 34
## 0.63  : 37
## 1.0.0 : 38
## 1.3.0 : 41
## 1.6.0 : 45
## 2.0.0 : 45


Number of base objects:		1324
Number of functions in base:	1282
	 starting with 'is.' :	  50


In [2]:
## Do we have a method (probably)?
is.method <- function(fname) {
    isFun <- function(name) (exists(name, mode="function") &&
                             is.na(match(name, c("is", "as"))))
    np <- length(sp <- strsplit(fname, split = "\\.")[[1]])
    if(np <= 1 )
        FALSE
    else
        (isFun(paste(sp[1:(np-1)], collapse = '.')) ||
         (np >= 3 &&
          isFun(paste(sp[1:(np-2)], collapse = '.'))))
}

In [3]:
is.ALL <- function(obj, func.names = ls(pos=length(search())),
		   not.using = c("is.single", "is.real", "is.loaded",
                     "is.empty.model", "is.R", "is.element", "is.unsorted"),
		   true.only = FALSE, debug = FALSE)
{
    ## Purpose: show many 'attributes' of  R object __obj__
    ## -------------------------------------------------------------------------
    ## Arguments: obj: any R object
    ## -------------------------------------------------------------------------
    ## Author: Martin Maechler, Date: 6 Dec 1996

    is.fn <- func.names[substring(func.names,1,3) == "is."]
    is.fn <- is.fn[substring(is.fn,1,7) != "is.na<-"]
    use.fn <- is.fn[ is.na(match(is.fn, not.using))
                    & ! sapply(is.fn, is.method) ]

    r <- if(true.only) character(0)
    else structure(vector("list", length= length(use.fn)), names= use.fn)
    for(f in use.fn) {
	if(any(f == c("is.na", "is.finite"))) {
	    if(!is.list(obj) && !is.vector(obj) && !is.array(obj)) {
		if(!true.only) r[[f]] <- NA
		next
	    }
	}
	if(any(f == c("is.nan", "is.finite", "is.infinite"))) {
	    if(!is.atomic(obj)) {
	    	if(!true.only) r[[f]] <- NA
	    	next
	    }
	}
	if(debug) cat(f,"")
	fn <- get(f)
	rr <- if(is.primitive(fn) || length(formals(fn))>0)  fn(obj) else fn()
	if(!is.logical(rr)) cat("f=",f," --- rr	 is NOT logical	 = ",rr,"\n")
	##if(1!=length(rr))   cat("f=",f," --- rr NOT of length 1; = ",rr,"\n")
	if(true.only && length(rr)==1 && !is.na(rr) && rr) r <- c(r, f)
	else if(!true.only) r[[f]] <- rr
    }
    if(debug)cat("\n")
    if(is.list(r)) structure(r, class = "isList") else r
}

In [4]:
print.isList <- function(x, ..., verbose = getOption("verbose"))
{
    ## Purpose:	 print METHOD  for `isList' objects
    ## ------------------------------------------------
    ## Author: Martin Maechler, Date: 12 Mar 1997
    if(is.list(x)) {
        if(verbose) cat("print.isList(): list case (length=",length(x),")\n")
	nm <- format(names(x))
	rr <- lapply(x, stats::symnum, na = "NA")
	for(i in seq_along(x)) cat(nm[i],":",rr[[i]],"\n", ...)
    } else NextMethod("print", ...)
}

In [5]:
is.ALL(NULL)
##fails: is.ALL(NULL, not.using = c("is.single", "is.loaded"))

is.array           : . 
is.atomic          : | 
is.call            : . 
is.character       : . 
is.complex         : . 
is.data.frame      : . 
is.double          : . 
is.environment     : . 
is.expression      : . 
is.factor          : . 
is.finite          : NA 
is.function        : . 
is.infinite        :  
is.integer         : . 
is.language        : . 
is.list            : . 
is.logical         : . 
is.matrix          : . 
is.na              : NA 
is.name            : . 
is.nan             :  
is.null            : | 
is.numeric         : . 
is.numeric_version : . 
is.object          : . 
is.ordered         : . 
is.package_version : . 
is.pairlist        : | 
is.primitive       : . 
is.qr              : . 
is.raw             : . 
is.recursive       : . 
is.symbol          : . 
is.table           : . 
is.vector          : . 

In [6]:
is.ALL(NULL,   true.only = TRUE)

[1] "is.atomic"   "is.null"     "is.pairlist"

In [7]:
all.equal(NULL, pairlist())
## list() != NULL == pairlist() :

[1] TRUE

In [8]:
is.ALL(list(), true.only = TRUE)

[1] "is.list"      "is.recursive" "is.vector"

In [9]:
(pl <- is.ALL(pairlist(1,    list(3,"A")), true.only = TRUE))

[1] "is.list"      "is.pairlist"  "is.recursive"

In [10]:
(ll <- is.ALL(    list(1,pairlist(3,"A")), true.only = TRUE))

[1] "is.list"      "is.recursive" "is.vector"

In [11]:
all.equal(pl[pl != "is.pairlist"],
          ll[ll != "is.vector"])## TRUE

[1] TRUE

In [12]:
is.ALL(1:5)

is.array           : . 
is.atomic          : | 
is.call            : . 
is.character       : . 
is.complex         : . 
is.data.frame      : . 
is.double          : . 
is.environment     : . 
is.expression      : . 
is.factor          : . 
is.finite          : | | | | | 
is.function        : . 
is.infinite        : . . . . . 
is.integer         : | 
is.language        : . 
is.list            : . 
is.logical         : . 
is.matrix          : . 
is.na              : . . . . . 
is.name            : . 
is.nan             : . . . . . 
is.null            : . 
is.numeric         : | 
is.numeric_version : . 
is.object          : . 
is.ordered         : . 
is.package_version : . 
is.pairlist        : . 
is.primitive       : . 
is.qr              : . 
is.raw             : . 
is.recursive       : . 
is.symbol          : . 
is.table           : . 
is.vector          : | 

In [13]:
is.ALL(array(1:24, 2:4))

is.array           : | 
is.atomic          : | 
is.call            : . 
is.character       : . 
is.complex         : . 
is.data.frame      : . 
is.double          : . 
is.environment     : . 
is.expression      : . 
is.factor          : . 
is.finite          : | | | | | | | | | | | | | | | | | | | | | | | | 
is.function        : . 
is.infinite        : . . . . . . . . . . . . . . . . . . . . . . . . 
is.integer         : | 
is.language        : . 
is.list            : . 
is.logical         : . 
is.matrix          : . 
is.na              : . . . . . . . . . . . . . . . . . . . . . . . . 
is.name            : . 
is.nan             : . . . . . . . . . . . . . . . . . . . . . . . . 
is.null            : . 
is.numeric         : | 
is.numeric_version : . 
is.object          : . 
is.ordered         : . 
is.package_version : . 
is.pairlist        : . 
is.primitive       : . 
is.qr              : . 
is.raw             : . 
is.recursive       : . 
is.symbol          : . 
is.table           : . 


In [14]:
is.ALL(1 + 3)

is.array           : . 
is.atomic          : | 
is.call            : . 
is.character       : . 
is.complex         : . 
is.data.frame      : . 
is.double          : | 
is.environment     : . 
is.expression      : . 
is.factor          : . 
is.finite          : | 
is.function        : . 
is.infinite        : . 
is.integer         : . 
is.language        : . 
is.list            : . 
is.logical         : . 
is.matrix          : . 
is.na              : . 
is.name            : . 
is.nan             : . 
is.null            : . 
is.numeric         : | 
is.numeric_version : . 
is.object          : . 
is.ordered         : . 
is.package_version : . 
is.pairlist        : . 
is.primitive       : . 
is.qr              : . 
is.raw             : . 
is.recursive       : . 
is.symbol          : . 
is.table           : . 
is.vector          : | 

In [15]:
e13 <- expression(1 + 3)
is.ALL(e13)

Warning message in fn(obj):
“is.na() applied to non-(list or vector) of type 'expression'”

is.array           : . 
is.atomic          : . 
is.call            : . 
is.character       : . 
is.complex         : . 
is.data.frame      : . 
is.double          : . 
is.environment     : . 
is.expression      : | 
is.factor          : . 
is.finite          : NA 
is.function        : . 
is.infinite        : NA 
is.integer         : . 
is.language        : | 
is.list            : . 
is.logical         : . 
is.matrix          : . 
is.na              : . 
is.name            : . 
is.nan             : NA 
is.null            : . 
is.numeric         : . 
is.numeric_version : . 
is.object          : . 
is.ordered         : . 
is.package_version : . 
is.pairlist        : . 
is.primitive       : . 
is.qr              : . 
is.raw             : . 
is.recursive       : | 
is.symbol          : . 
is.table           : . 
is.vector          : | 

In [16]:
is.ALL(substitute(expression(a + 3), list(a=1)), true.only = TRUE)

[1] "is.call"      "is.language"  "is.recursive"

In [17]:
is.ALL(y ~ x) #--> NA	 for is.na & is.finite

is.array           : . 
is.atomic          : . 
is.call            : | 
is.character       : . 
is.complex         : . 
is.data.frame      : . 
is.double          : . 
is.environment     : . 
is.expression      : . 
is.factor          : . 
is.finite          : NA 
is.function        : . 
is.infinite        : NA 
is.integer         : . 
is.language        : | 
is.list            : . 
is.logical         : . 
is.matrix          : . 
is.na              : NA 
is.name            : . 
is.nan             : NA 
is.null            : . 
is.numeric         : . 
is.numeric_version : . 
is.object          : | 
is.ordered         : . 
is.package_version : . 
is.pairlist        : . 
is.primitive       : . 
is.qr              : . 
is.raw             : . 
is.recursive       : | 
is.symbol          : . 
is.table           : . 
is.vector          : . 

In [18]:
is0 <- is.ALL(numeric(0))
is0.ok <- 1 == (lis0 <- sapply(is0, length))
is0[!is0.ok]
is0 <- unlist(is0)
is0

$is.finite
logical(0)

$is.infinite
logical(0)

$is.na
logical(0)

$is.nan
logical(0)

is.array          is.atomic            is.call       is.character 
             FALSE               TRUE              FALSE              FALSE 
        is.complex      is.data.frame          is.double     is.environment 
             FALSE              FALSE               TRUE              FALSE 
     is.expression          is.factor        is.function         is.integer 
             FALSE              FALSE              FALSE              FALSE 
       is.language            is.list         is.logical          is.matrix 
             FALSE              FALSE              FALSE              FALSE 
           is.name            is.null         is.numeric is.numeric_version 
             FALSE              FALSE               TRUE              FALSE 
         is.object         is.ordered is.package_version        is.pairlist 
             FALSE              FALSE              FALSE              FALSE 
      is.primitive              is.qr             is.raw       is.recursive 
             FALSE              FALSE              FALSE              FALSE 
         is.symbol           is.table          is.vector 
             FALSE              FALSE               TRUE

In [19]:
ispi <- unlist(is.ALL(pi))
all(ispi[is0.ok] == is0)

[1] TRUE

In [20]:
is.ALL(numeric(0), true=TRUE)

[1] "is.atomic"  "is.double"  "is.numeric" "is.vector"

In [21]:
is.ALL(array(1,1:3), true=TRUE)

[1] "is.array"   "is.atomic"  "is.double"  "is.numeric"

In [22]:
is.ALL(cbind(1:3), true=TRUE)

[1] "is.array"   "is.atomic"  "is.integer" "is.matrix"  "is.numeric"

In [23]:
is.ALL(structure(1:7, names = paste("a",1:7,sep="")))

is.array           : . 
is.atomic          : | 
is.call            : . 
is.character       : . 
is.complex         : . 
is.data.frame      : . 
is.double          : . 
is.environment     : . 
is.expression      : . 
is.factor          : . 
is.finite          : | | | | | | | 
is.function        : . 
is.infinite        : . . . . . . . 
is.integer         : | 
is.language        : . 
is.list            : . 
is.logical         : . 
is.matrix          : . 
is.na              : . . . . . . . 
is.name            : . 
is.nan             : . . . . . . . 
is.null            : . 
is.numeric         : | 
is.numeric_version : . 
is.object          : . 
is.ordered         : . 
is.package_version : . 
is.pairlist        : . 
is.primitive       : . 
is.qr              : . 
is.raw             : . 
is.recursive       : . 
is.symbol          : . 
is.table           : . 
is.vector          : | 

In [24]:
is.ALL(structure(1:7, names = paste("a",1:7,sep="")), true.only = TRUE)

[1] "is.atomic"  "is.integer" "is.numeric" "is.vector"

In [25]:
x <- 1:20 ; y <- 5 + 6*x + rnorm(20)
lm.xy <- lm(y ~ x)
is.ALL(lm.xy)

is.array           : . 
is.atomic          : . 
is.call            : . 
is.character       : . 
is.complex         : . 
is.data.frame      : . 
is.double          : . 
is.environment     : . 
is.expression      : . 
is.factor          : . 
is.finite          : NA 
is.function        : . 
is.infinite        : NA 
is.integer         : . 
is.language        : . 
is.list            : | 
is.logical         : . 
is.matrix          : . 
is.na              : . . . . . . . . . . . . 
is.name            : . 
is.nan             : NA 
is.null            : . 
is.numeric         : . 
is.numeric_version : . 
is.object          : | 
is.ordered         : . 
is.package_version : . 
is.pairlist        : . 
is.primitive       : . 
is.qr              : . 
is.raw             : . 
is.recursive       : | 
is.symbol          : . 
is.table           : . 
is.vector          : . 

In [26]:
is.ALL(structure(1:7, names = paste("a",1:7,sep="")))

is.array           : . 
is.atomic          : | 
is.call            : . 
is.character       : . 
is.complex         : . 
is.data.frame      : . 
is.double          : . 
is.environment     : . 
is.expression      : . 
is.factor          : . 
is.finite          : | | | | | | | 
is.function        : . 
is.infinite        : . . . . . . . 
is.integer         : | 
is.language        : . 
is.list            : . 
is.logical         : . 
is.matrix          : . 
is.na              : . . . . . . . 
is.name            : . 
is.nan             : . . . . . . . 
is.null            : . 
is.numeric         : | 
is.numeric_version : . 
is.object          : . 
is.ordered         : . 
is.package_version : . 
is.pairlist        : . 
is.primitive       : . 
is.qr              : . 
is.raw             : . 
is.recursive       : . 
is.symbol          : . 
is.table           : . 
is.vector          : | 

In [27]:
is.ALL(structure(1:7, names = paste("a",1:7,sep="")), true.only = TRUE)

[1] "is.atomic"  "is.integer" "is.numeric" "is.vector"